
Задание 1.
Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>


In [2]:
import requests

In [3]:
import pandas as pd

In [4]:
from bs4 import BeautifulSoup

In [50]:
KEYWORDS =  'return'

In [51]:
res = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(res.text)

In [52]:
posts = soup.find_all('article', class_='tm-articles-list__item')
posts

[<article class="tm-articles-list__item" data-navigatable="" id="569448" tabindex="0"><div class="tm-article-snippet"><div class="tm-article-snippet__meta-container"><div class="tm-article-snippet__meta"><span class="tm-user-info tm-article-snippet__author"><a class="tm-user-info__userpic" href="/ru/users/VitalyCherkov/" title="VitalyCherkov"><div class="tm-entity-image"><img alt="" class="tm-entity-image__pic" height="24" loading="lazy" src="//habrastorage.org/r/w32/getpro/habr/avatars/943/093/6f0/9430936f0cf1c1b476000528a1711e74.png" width="24"/></div></a><span class="tm-user-info__user"><a class="tm-user-info__username" href="/ru/users/VitalyCherkov/">
       VitalyCherkov
     </a></span></span><span class="tm-article-snippet__datetime-published"><time datetime="2021-07-26T14:02:01.000Z" title="2021-07-26, 17:02">сегодня в 17:02</time></span></div><!-- --></div><h2 class="tm-article-snippet__title tm-article-snippet__title_h2"><a class="tm-article-snippet__title-link" data-article-

In [17]:
# добираемся до текста со ссылкой
articles_intro = list(map(lambda x: x.find('h2', class_='tm-article-snippet__title'), posts))

In [19]:
# добираемся до ссылок
a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))

In [21]:
# формируем полноценные ссылки
all_refs = list(map(lambda x: 'https://habr.com' + x, a_list))

In [43]:
date = list(map(lambda x: x.find('span', class_='tm-article-snippet__datetime-published'), posts))


In [55]:
kom_news = pd.DataFrame()
for post in posts:
    if KEYWORDS in post:
        date = soup.find('span', class_='tm-article-snippet__datetime-published').text
        articles_intro = soup.find('h2', class_='tm-article-snippet__title').text    
        w = title_element.attrs.get('href')
        row = {'date': date, 'title': articles_intro, 'text': w}
        kom_news = pd.concat([kom_news, pd.DataFrame([row])])  
kom_news

""



Задание 2.
Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.


In [1]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [2]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']

In [3]:
payload = {
'emailAddresses': EMAIL
}

In [4]:
headers = {
'Vaar-Header-App-Build-Version': '1.0.0',
'Vaar-Header-App-Product': 'hackcheck-web-avast',
'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
'Vaar-Version': '0'
}

In [8]:
req = requests.post(URL, json=payload, headers=headers)

In [9]:
req

<Response [200]>

In [12]:
req.json()

{'breaches': {'2': {'breachId': 2,
   'site': 'linkedin.com',
   'recordsCount': 158591429,
   'description': "In 2012, online professional networking platform LinkedIn fell victim to a breach of its members' passwords. Four years later, 117 million email and password combinations from that breach appeared for sale on a dark web marketplace. \n\nThe leaked passwords had only been protected by unsalted SHA-1 cryptographic hashing, which prompted LinkedIn to enforce salted hashing after the breach. Russian national Yevgeniy Nikulin was accused of the breach and was extradited from the Czech Republic to the United States as of March 2018.",
   'publishDate': '2016-10-21T00:00:00Z',
   'statistics': {'usernames': 0,
    'passwords': 111975337,
    'emails': 158591429}},
  '3': {'breachId': 3,
   'site': 'adobe.com',
   'recordsCount': 152046506,
   'description': "In October of 2013, criminals penetrated Adobe's corporate network and the stole source code for several of its software produc

In [17]:
avast = pd.DataFrame(req.json()['breaches'])

In [20]:
row1 = avast.iloc[[4, 1, 3]] 
row1

,2,3,12,15,41,2961,3164,3176,3520,3587,3669,13094,13662,16768,17009,17110,17670,37177
publishDate,2016-10-21T00:00:00Z,2016-10-21T00:00:00Z,2016-10-29T00:00:00Z,2016-10-23T00:00:00Z,2016-10-24T00:00:00Z,2017-01-31T00:00:00Z,2017-02-14T00:00:00Z,2017-02-14T00:00:00Z,2017-03-15T00:00:00Z,2017-03-01T00:00:00Z,2017-03-24T00:00:00Z,2017-11-04T00:00:00Z,2018-02-18T00:00:00Z,2019-06-13T00:00:00Z,2019-10-17T00:00:00Z,2020-01-03T00:00:00Z,2020-05-28T00:00:00Z,2021-02-11T00:00:00Z
site,linkedin.com,adobe.com,vk.com,imesh.com,dropbox.com,cdprojektred.com,cfire.mail.ru,parapa.mail.ru,globalreach.eu,rayli.com.cn,youku.com,myheritage.com,netlog.com,canva.com,zynga.com,azcentral.com,wishbone.io,forums.vkmonline.com
description,"In 2012, online professional networking platfo...","In October of 2013, criminals penetrated Adobe...",Popular Russian social networking platform VKo...,"In June 2016, a cache of over 51 million user ...",Cloud storage company Dropbox suffered a major...,"In March 2016, CDProjektRed.com.com's forum da...","In July and August of 2016, two criminals carr...","In July and August 2016, two criminals execute...","In 2016, Global Reach Technology's database wa...","On an unconfirmed date, Chinese gossip site Ra...",Youku is a large Chinese video content company...,"In October 2017, a customer database belonging...",Netlog (formerly known as Facebox and Bingbox)...,"In May 2019, graphic-design site Canva's datab...","In September 2019, the game developer Zynga wa...","At an unconfirmed date, online Arizona newspap...","In January 2020, the online poll website Wishb...","At an unconfirmed date, the Russian-language m..."
